In [1]:
import ogr,pandas

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [4]:
rdd = sc.textFile("./data/dltb.tsv")

## 过滤掉header

In [5]:
rdd.take(1)

['FID\tSHAPE\tTBBH\tDLMC\tTBMJ\tShape_Leng\tShape_Area']

In [6]:
geoRdd = rdd.filter(lambda line : line.find("POLYGON")>0)

## 借用pandas查看第一条记录

In [7]:
pandas.DataFrame(geoRdd.take(1)[0].split("\t"))

,0
0,0
1,"POLYGON ((11135971.0803 2710491.2501,11135968...."
2,379
3,旱地
4,1489.7
5,1375.76858615
6,47227.8152174


## map操作，设计key为地类名称，value为图斑面积

In [8]:
mapRdd = geoRdd.map(lambda line:(line.split("\t")[3],
                             float(line.split("\t")[4])))

## 查看一下前10条的内容，可以看见数据中的地类名称和图斑面积被构建成了一个map

In [9]:
print(mapRdd.take(10))

[('旱地', 1489.7), ('园地', 174938.39), ('林地', 16361.45), ('园地', 6216.62), ('园地', 767.08), ('园地', 84939.96), ('园地', 808.57), ('园地', 26997.04), ('旱地', 6367.33), ('林地', 23087.05)]


## 进行reduceByKey操作

In [10]:
res = mapRdd.reduceByKey(lambda x,y:x+y).collect()

In [11]:
print(res)

[('旱地', 501681.6499999999), ('园地', 1281049.4700000002), ('农村居民点用地', 82505.71), ('其他独立建设用地', 2507353.24), ('林地', 1648852.08)]


In [12]:
rdd.filter(lambda line : line.find("POLYGON")>0)\
.map(lambda line:(line.split("\t")[3],
                float(line.split("\t")[4])))\
.reduceByKey(lambda x,y:x+y).collect()

[('旱地', 501681.6499999999),
 ('园地', 1281049.4700000002),
 ('农村居民点用地', 82505.71),
 ('其他独立建设用地', 2507353.24),
 ('林地', 1648852.08)]